
# Importing the required Libraries

In [1]:
import os
import re
import sys
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
import datetime
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

IS_OFFLine = False

In [2]:
## 统计缺失值函数

def na_summary(dat):
    result = pd.isnull(dat).sum().to_frame()
    result.rename(columns={0:'count'}, inplace=True)
    result['percent'] = (result['count'] / len(dat)).map(lambda x: format(x, '.1%'))
    print(dat.shape, '\n', '========================', '\n', result)
    
def xgb_valid(train_set_x,train_set_y):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'silent':1,
              'nthread':8
              }
    dtrain = xgb.DMatrix(train_set_x, label=train_set_y)
    model = xgb.cv(params, dtrain, num_boost_round=1000,nfold=5,metrics={'auc'},seed=10)
    print(model)


def xgb_feature(train_set_x,train_set_y,test_set_x,test_set_y):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'silent':1
              }
    dtrain = xgb.DMatrix(train_set_x, label=train_set_y)
    dvali = xgb.DMatrix(test_set_x)
    model = xgb.train(params, dtrain, num_boost_round=800)
    predict = model.predict(dvali)
    return predict, model

# Preprocessing Data Set

## train_auth_info

In [ ]:
## 变量构建

# 统计“手机号”是否缺失（缺失为1，否则为0）      train_auth_phone_na   
# 统计“身份证号”是否缺失（缺失为1，否则为0）    train_auth_id_card_na

In [3]:
train_auth = pd.read_csv('AI_risk_train_V3.0/train_auth_info.csv', parse_dates = ['auth_time'])

train_auth['train_auth_phone_na']   = train_auth['phone'].map(lambda x: 0 if str(x) == 'nan' else 1)
train_auth['train_auth_id_card_na'] = train_auth['id_card'].map(lambda x: 0 if str(x) == 'nan' else 1)

train_auth.head(3)

,id_card,auth_time,phone,id,train_auth_phone_na,train_auth_id_card_na
0,4****************0,2017-06-10,132****2008,501951980776722440,1,1
1,NaN,NaT,187*****500,525890212484616200,1,0
2,5****************9,2015-10-10,135****3522,599309364691472392,1,1


## train_bankcard_info

In [ ]:
## 变量重塑

# 统计每个id对应的“银行名称”的个数                train_bankcard_bankname_count
# 统计每个id对应的“银行名称（去重）”的个数        train_bankcard_bankname_nunique
# 统计每个id对应的“银行卡类型（去重）”的个数      train_bankcard_cardtype_nunique
# 统计每个id对应的“手机号（去重）”的个数          train_bankcard_phone_nunique
# 统计每个id对应的“银行卡号后四位（去重）”的个数  train_bankcard_tailnum_nunique    不明白这个变量的意义【删除】
# 统计每个id对应的“银行卡绑定手机号”列表          train_bankcard_phone_list         目的是判断当前手机是否在手机号列表中

In [4]:
train_bankcard = pd.read_csv('AI_risk_train_V3.0/train_bankcard_info.csv')

train_bankcard_bankname_count  =train_bankcard.groupby(by='id',as_index=False)['bank_name'].agg({'train_bankcard_bank_name_count': len})
train_bankcard_bankname_nunique=train_bankcard.groupby(by='id',as_index=False)['bank_name'].agg({'train_bankcard_bank_name_nunique': 'nunique'})
train_bankcard_cardtype_nunique=train_bankcard.groupby(by='id',as_index=False)['card_type'].agg({'train_bankcard_card_type_nunique': 'nunique'})
train_bankcard_phone_nunique   =train_bankcard.groupby(by='id',as_index=False)['phone'].agg({'train_bankcard_phone_nunique': 'nunique'})
train_bankcard_phone_list=train_bankcard.groupby(by='id',as_index=False)['phone'].agg({'train_bankcard_phone_list': lambda x: list(set(x.tolist()))})
# train_bankcard_tailnum_nunique =train_bankcard.groupby(by='id',as_index=False)['tail_num'].agg({'train_bankcard_tailnum_nunique': 'nunique'})

## train_creat_info

In [ ]:
## 变量构建

# 最高分-当前分数          train_credit_score_inverse
# 网购平台信用额度可用值   train_credit_can_use

In [5]:
train_credit = pd.read_csv('AI_risk_train_V3.0/train_credit_info.csv')

credit_score_max = max(train_credit.credit_score)
train_credit['train_credit_score_inverse'] = train_credit['credit_score'].map(lambda x: credit_score_max-x)
train_credit['train_credit_can_use'] = train_credit['quota'] - train_credit['overdraft']
train_credit['train_credit_can_use_ratio'] = train_credit['overdraft'] / (train_credit['quota'] + 0.01)

train_credit.head(3)

,credit_score,overdraft,quota,id,train_credit_score_inverse,train_credit_can_use,train_credit_can_use_ratio
0,549.0,0.0,0.0,563888070781309192,56.0,0.0,0.000000
1,400.0,0.0,0.0,591567740590887176,205.0,0.0,0.000000
2,490.0,4955.9,5083.0,464888846169936136,115.0,127.1,0.974993


## train_order_info

In [ ]:
## 变量构建

# 规范化“下单时间”
# 统计每个id对应的最晚下单时间            train_order_time_max
# 统计每个id对应的最早下单时间            train_order_time_min
# 统计每个id对应“在线支付”的次数        train_order_type_pay_zaixian
# 统计每个id对应“货到付款”的次数        train_order_type_pay_huodao
# 统计每个id对应“购物”的次数            train_order_id_count
# 统计每个id对应“订单金额”的均值        train_order_amt_order_mean
# 统计每个id对应“商品单价”的均值        train_order_unit_price_mean
# 统计每个id对应“下单”（去重）的次数    train_order_order_time_nunique   不明白其意义【删除】

In [6]:
warnings.filterwarnings('ignore')

train_order = pd.read_csv('AI_risk_train_V3.0/train_order_info.csv', parse_dates=['time_order'])

train_order['time_order'] = train_order['time_order'] \
    .map(lambda x : pd.lib.NaT if (str(x) == '0' or x == 'NA' or x == 'nan')
         else (datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') if ':' in str(x)
               else (datetime.datetime.utcfromtimestamp(int(x[0:10])) + datetime.timedelta(hours = 8))))

train_order_time_max = train_order.groupby(by='id', as_index=False)['time_order'].agg({'train_order_time_max': max})
train_order_time_min = train_order.groupby(by='id', as_index=False)['time_order'].agg({'train_order_time_min': min})

train_order_type_zaixian=train_order.groupby(by='id', as_index=False)['type_pay'].agg({'train_order_type_pay_zaixian': lambda x: sum(x == '在线支付')})
train_order_type_huodao =train_order.groupby(by='id', as_index=False)['type_pay'].agg({'train_order_type_pay_huodao': lambda x: sum(x == '货到付款')})

train_order_id_count = train_order.groupby(by = 'id', as_index=False)['id'].agg({'train_order_id_count': len})

train_order_amt_order_mean  = train_order.groupby(by = 'id',as_index=False)['amt_order'].agg({'train_order_amt_order_mean': np.mean})
train_order_unit_price_mean = train_order.groupby(by = 'id',as_index=False)['unit_price'].agg({'train_order_unit_price_mean': np.mean})
# train_order_order_time_nunique = train_order.groupby(by = 'id',as_index=False)['time_order'].agg({'train_order_order_time_nunique': 'nunique'})

## train_recieve

In [ ]:
## 变量构建

# 独热编码地区                                      train_recieve_new
# 统计每个id对应的“固定收货手机号”的个数          train_recieve_phone_count   即此人固定收货手机号收货的次数
# 统计每个id对应的“固定收货手机号”的个数（去重）  train_recieve_phone_nunique 即此人有多少固定收货手机号

In [8]:
train_recieve = pd.read_csv('AI_risk_train_V3.0/train_recieve_addr_info.csv')

train_recieve['region'] = train_recieve['region'].map(lambda x: str(x)[:2])
train_recieve_new = pd.crosstab(train_recieve.id, train_recieve.region).reset_index()

train_recieve_phone_count   = train_recieve.groupby(by=['id'])['fix_phone'].agg({'train_recieve_phone_count': len})
train_recieve_phone_nunique = train_recieve.groupby(by=['id'])['fix_phone'].agg({'train_recieve_phone_nunique': 'nunique'})

## train_target

In [9]:
train_target = pd.read_csv('AI_risk_train_V3.0/train_target.csv', parse_dates = ['appl_sbm_tm'])

train_target['hour']  = train_target['appl_sbm_tm'].map(lambda x: x.hour)
train_target['month'] = train_target['appl_sbm_tm'].map(lambda x: x.month)
train_target['year']  = train_target['appl_sbm_tm'].map(lambda x: x.year)

## train_user

In [ ]:
## 变量构建

# 统计兴趣变量是否缺失         is_hobby_na
# 统计身份证号是否缺失         is_id_card_na
# 规范化生日变量

In [10]:
train_user = pd.read_csv('AI_risk_train_V3.0/train_user_info.csv')

train_user['train_user_is_hobby_na'] = train_user['hobby'].map(lambda x: 0 if str(x) == 'nan' else 1)
train_user['train_user_is_id_card_na'] = train_user['id_card'].map(lambda x:0 if str(x )== 'nan' else 1)


tmp = train_user[['id','birthday']].set_index(['id'])

train_user_is_double_ = tmp['birthday'].map(lambda x: (str(x) == '--')*1).reset_index(name='train_user_is_double_')
train_user_is_0_0_0 = tmp['birthday'].map(lambda x:(str(x) == '0-0-0')*1).reset_index(name='train_user_is_0_0_0')
train_user_is_1_1_1 = tmp['birthday'].map(lambda x:(str(x) == '1-1-1')*1).reset_index(name='train_user_is_1_1_1')
train_user_is_0000_00_00 = tmp['birthday'].map(lambda x:(str(x) == '0000-00-00')*1).reset_index(name='train_user_is_0000_00_00')
train_user_is_0001_1_1 = tmp['birthday'].map(lambda x:(str(x) == '0001-1-1')*1).reset_index(name='train_user_is_0001_1_1')
train_user_is_hou_in = tmp['birthday'].map(lambda x:('后' in str(x))*1).reset_index(name='train_user_is_hou_in')

train_user['birthday'] = train_user['birthday'] \
    .map(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d') 
         if(re.match('19\d{2}-\d{1,2}-\d{1,2}', str(x)) and '-0' not in str(x)) else pd.lib.NaT)

## 合并数据

In [11]:
train_data = pd.merge(train_target, train_auth, on=['id'], how='left')
train_data = pd.merge(train_data, train_user, on=['id'], how='left')
train_data = pd.merge(train_data, train_credit,on=['id'], how='left')
train_data = pd.merge(train_data, train_bankcard_phone_list, on=['id'], how='left')

train_data = pd.merge(train_data,train_bankcard_bankname_count,on=['id'],how='left')
train_data = pd.merge(train_data,train_bankcard_bankname_nunique,on=['id'],how='left')
train_data = pd.merge(train_data,train_bankcard_cardtype_nunique,on=['id'],how='left')
train_data = pd.merge(train_data,train_bankcard_phone_nunique,on=['id'],how='left')

train_data = pd.merge(train_data,train_order_time_max,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_time_min,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_type_zaixian,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_type_huodao,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_id_count,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_amt_order_mean,on=['id'],how='left')
train_data = pd.merge(train_data,train_order_unit_price_mean,on=['id'],how='left')

train_data = pd.merge(train_data,train_recieve_new,on=['id'],how='left')
train_data = pd.merge(train_data,train_recieve_phone_count,on=['id'],how='left')
train_data = pd.merge(train_data,train_recieve_phone_nunique,on=['id'],how='left')

train_data = pd.merge(train_data,train_user_is_double_,on=['id'],how='left')
train_data = pd.merge(train_data,train_user_is_0_0_0,on=['id'],how='left')
train_data = pd.merge(train_data,train_user_is_1_1_1,on=['id'],how='left')
train_data = pd.merge(train_data,train_user_is_0000_00_00,on=['id'],how='left')
train_data = pd.merge(train_data,train_user_is_0001_1_1,on=['id'],how='left')
train_data = pd.merge(train_data,train_user_is_hou_in,on=['id'],how='left')

In [ ]:
## 变量重塑

# 统计每行缺失值的个数                            train_na_num
# 统计auth_info和user_info里面的id_card是否一致   train_data_same_id_card

In [12]:
train_data['train_na_num'] = train_data.isnull().sum(axis=1)

# create "the_same_idcard"  

auth_idcard = list(train_data['id_card_x'])
user_idcard = list(train_data['id_card_y'])

idcard_result = []
for indexx, uu in enumerate(auth_idcard):
    
    if (str(auth_idcard[indexx])=='nan') and (str(user_idcard[indexx])=='nan'):
        idcard_result.append(0)
        
    elif (str(auth_idcard[indexx])!='nan') and (str(user_idcard[indexx])=='nan'):
        idcard_result.append(1)
        
    elif (str(auth_idcard[indexx])=='nan') and (str(user_idcard[indexx])!='nan'):
        idcard_result.append(2)
        
    else:
        ttt1 = str(auth_idcard[indexx])[0] + str(auth_idcard[indexx])[-1]
        ttt2 = str(user_idcard[indexx])[0] + str(user_idcard[indexx])[-1]
        
        if ttt1 == ttt2:
            idcard_result.append(3)
        if ttt1 != ttt2:
            idcard_result.append(4)
            
train_data['train_data_same_id_card'] = idcard_result


# 判断当前的手机号(train_auth)是否在手机号列表中，并剔除手机号列表
train_data['train_data_exist_phone'] = train_data.apply(lambda x: x['phone'] in x['train_bankcard_phone_list'], axis=1)
train_data['train_data_exist_phone'] = train_data['train_data_exist_phone']*1
train_data = train_data.drop(['train_bankcard_phone_list'], axis=1)

train_data['train_data_diff_day'] = train_data.apply(lambda x: (x['appl_sbm_tm'] - x['auth_time']).days, axis=1) # 贷款提交时间-认证时间
train_data['train_data_age'] = train_data.apply(lambda x: (x['appl_sbm_tm'] - x['birthday']).days / 365, axis=1) # 用户年龄

train_data['train_data_day_order_max'] = train_data.apply(lambda x: (x['appl_sbm_tm'] - x['train_order_time_max']).days, axis=1)
train_data = train_data.drop(['train_order_time_max'], axis=1)

train_data['train_data_day_order_min'] = train_data.apply(lambda row: (row['appl_sbm_tm'] - row['train_order_time_min']).days,axis=1)
train_data = train_data.drop(['train_order_time_min'],axis=1)

In [ ]:
train_data.columns

In [13]:
if IS_OFFLine == False:
    train_data = train_data.drop(['appl_sbm_tm','id','id_card_x','auth_time','phone','birthday','hobby','id_card_y'],axis=1)

# if IS_OFFLine == True:
#     dummy_fea = ['sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound','account_grade','industry']
#     dummy_df = pd.get_dummies(train_data.loc[:,dummy_fea])
#     train_data_copy = pd.concat([train_data,dummy_df],axis=1)
#     train_data_copy = train_data_copy.fillna(0)
#     vaild_train_data = train_data_copy.drop(dummy_fea,axis=1)
#     valid_train_train = vaild_train_data[vaild_train_data.appl_sbm_tm < datetime.datetime(2017,4,1)]
#     valid_train_test = vaild_train_data[vaild_train_data.appl_sbm_tm >= datetime.datetime(2017,4,1)]
#     valid_train_train = valid_train_train.drop(['appl_sbm_tm','id','id_card_x','auth_time','phone','birthday','hobby','id_card_y'],axis=1)
#     valid_train_test = valid_train_test.drop(['appl_sbm_tm','id','id_card_x','auth_time','phone','birthday','hobby','id_card_y'],axis=1)
#     vaild_train_x = valid_train_train.drop(['target'],axis=1)
#     vaild_test_x = valid_train_test.drop(['target'],axis=1)
#     redict_result, modelee = xgb_feature(vaild_train_x,valid_train_train['target'].values,vaild_test_x,None)
#     print('valid auc',roc_auc_score(valid_train_test['target'].values,redict_result))
#     sys.exit(23)

In [14]:
train_data.to_csv('train_data_all_1117.csv', encoding='gbk', index=False)

# test_data

## test_auth

In [15]:
test_auth = pd.read_csv('AI_Risk_data_Btest_V2.0/test_auth_info.csv',parse_dates = ['auth_time'])

test_auth['test_auth_id_card_na'] = test_auth['id_card'].map(lambda x: 0 if str(x) == 'nan' else 1)
test_auth['test_auth_phone_na'] = test_auth['phone'].map(lambda x: 0 if str(x) == 'nan' else 1)

test_auth['auth_time'].replace('0000-00-00', 'nan', inplace=True)
test_auth['auth_time'] = pd.to_datetime(test_auth['auth_time'])

## test_bankcard

In [16]:
test_bankcard = pd.read_csv('AI_Risk_data_Btest_V2.0/test_bankcard_info.csv')

test_bankcard_bankname_count  =test_bankcard.groupby(by='id',as_index=False)['bank_name'].agg({'test_bankcard_bank_name_count': len})
test_bankcard_bankname_nunique=test_bankcard.groupby(by='id',as_index=False)['bank_name'].agg({'test_bankcard_bank_name_nunique': 'nunique'})
test_bankcard_cardtype_nunique=test_bankcard.groupby(by='id',as_index=False)['card_type'].agg({'test_bankcard_card_type_nunique': 'nunique'})
test_bankcard_phone_nunique   =test_bankcard.groupby(by='id',as_index=False)['phone'].agg({'test_bankcard_phone_nunique': 'nunique'})
test_bankcard_phone_list=test_bankcard.groupby(by='id',as_index=False)['phone'].agg({'test_bankcard_phone_list': lambda x: list(set(x.tolist()))})

## test_credit

In [17]:
test_credit = pd.read_csv('AI_Risk_data_Btest_V2.0/test_credit_info.csv')

credit_score_max = max(test_credit.credit_score)
test_credit['test_credit_score_inverse'] = test_credit['credit_score'].map(lambda x: credit_score_max-x)
test_credit['test_credit_can_use'] = test_credit['quota'] - test_credit['overdraft']
test_credit['test_credit_can_use_ratio'] = test_credit['overdraft'] / (test_credit['quota'] + 0.01)

test_credit.head(3)

,credit_score,overdraft,quota,id,test_credit_score_inverse,test_credit_can_use,test_credit_can_use_ratio
0,389.0,0.0,0.0,647995096096051464,216.0,0.0,0.0
1,400.0,0.0,0.0,647548977264201736,205.0,0.0,0.0
2,384.0,0.0,0.0,639787275877617928,221.0,0.0,0.0


## test_order

In [18]:
warnings.filterwarnings('ignore')

test_order = pd.read_csv('AI_Risk_data_Btest_V2.0/test_order_info.csv', parse_dates=['time_order'])

test_order['time_order'] = test_order['time_order'] \
    .map(lambda x : pd.lib.NaT if (str(x) == '0' or x == 'NA' or x == 'nan')
         else (datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') if ':' in str(x)
               else (datetime.datetime.utcfromtimestamp(int(x[0:10])) + datetime.timedelta(hours = 8))))

test_order_time_max = test_order.groupby(by='id', as_index=False)['time_order'].agg({'test_order_time_max': max})
test_order_time_min = test_order.groupby(by='id', as_index=False)['time_order'].agg({'test_order_time_min': min})

test_order_type_zaixian=test_order.groupby(by='id', as_index=False)['type_pay'].agg({'test_order_type_pay_zaixian': lambda x: sum(x == '在线支付')})
test_order_type_huodao =test_order.groupby(by='id', as_index=False)['type_pay'].agg({'test_order_type_pay_huodao': lambda x: sum(x == '货到付款')})

test_order_id_count = test_order.groupby(by = 'id', as_index=False)['id'].agg({'test_order_id_count': len})

test_order_amt_order_mean  = test_order.groupby(by = 'id',as_index=False)['amt_order'].agg({'test_order_amt_order_mean': np.mean})
test_order_unit_price_mean = test_order.groupby(by = 'id',as_index=False)['unit_price'].agg({'test_order_unit_price_mean': np.mean})
# test_order_order_time_nunique = test_order.groupby(by = 'id',as_index=False)['time_order'].agg({'test_order_order_time_nunique': 'nunique'})

## test_recieve

In [19]:
test_recieve = pd.read_csv('AI_Risk_data_Btest_V2.0/test_recieve_addr_info.csv')

test_recieve['region'] = test_recieve['region'].map(lambda x: str(x)[:2])
test_recieve_new = pd.crosstab(test_recieve.id, test_recieve.region).reset_index()

test_recieve_phone_count   = test_recieve.groupby(by=['id'])['fix_phone'].agg({'test_recieve_phone_count': len})
test_recieve_phone_nunique = test_recieve.groupby(by=['id'])['fix_phone'].agg({'test_recieve_phone_nunique': 'nunique'})

## test_target

In [20]:
test_target = pd.read_csv('AI_Risk_data_Btest_V2.0/test_list.csv',parse_dates = ['appl_sbm_tm'])

test_target['hour']  = test_target['appl_sbm_tm'].map(lambda x: x.hour)
test_target['month'] = test_target['appl_sbm_tm'].map(lambda x: x.month)
test_target['year']  = test_target['appl_sbm_tm'].map(lambda x: x.year)

## test_user

In [21]:
test_user = pd.read_csv('AI_Risk_data_Btest_V2.0/test_user_info.csv',parse_dates = ['birthday'])

test_user['test_user_is_hobby_na'] = test_user['hobby'].map(lambda x: 0 if str(x) == 'nan' else 1)
test_user['test_user_is_id_card_na'] = test_user['id_card'].map(lambda x:0 if str(x )== 'nan' else 1)


tmp = test_user[['id','birthday']].set_index(['id'])

test_user_is_double_ = tmp['birthday'].map(lambda x: (str(x) == '--')*1).reset_index(name='test_user_is_double_')
test_user_is_0_0_0 = tmp['birthday'].map(lambda x:(str(x) == '0-0-0')*1).reset_index(name='test_user_is_0_0_0')
test_user_is_1_1_1 = tmp['birthday'].map(lambda x:(str(x) == '1-1-1')*1).reset_index(name='test_user_is_1_1_1')
test_user_is_0000_00_00 = tmp['birthday'].map(lambda x:(str(x) == '0000-00-00')*1).reset_index(name='test_user_is_0000_00_00')
test_user_is_0001_1_1 = tmp['birthday'].map(lambda x:(str(x) == '0001-1-1')*1).reset_index(name='test_user_is_0001_1_1')
test_user_is_hou_in = tmp['birthday'].map(lambda x:('后' in str(x))*1).reset_index(name='test_user_is_hou_in')

test_user['birthday'] = test_user['birthday'] \
    .map(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d') 
         if(re.match('19\d{2}-\d{1,2}-\d{1,2}', str(x)) and '-0' not in str(x)) else pd.lib.NaT)

## 合并数据

In [22]:
test_data = pd.merge(test_target, test_auth, on=['id'], how='left')
test_data = pd.merge(test_data, test_user, on=['id'], how='left')
test_data = pd.merge(test_data, test_credit,on=['id'], how='left')
test_data = pd.merge(test_data, test_bankcard_phone_list, on=['id'], how='left')

test_data = pd.merge(test_data,test_bankcard_bankname_count,on=['id'],how='left')
test_data = pd.merge(test_data,test_bankcard_bankname_nunique,on=['id'],how='left')
test_data = pd.merge(test_data,test_bankcard_cardtype_nunique,on=['id'],how='left')
test_data = pd.merge(test_data,test_bankcard_phone_nunique,on=['id'],how='left')

test_data = pd.merge(test_data,test_order_time_max,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_time_min,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_type_zaixian,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_type_huodao,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_id_count,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_amt_order_mean,on=['id'],how='left')
test_data = pd.merge(test_data,test_order_unit_price_mean,on=['id'],how='left')

test_data = pd.merge(test_data,test_recieve_new,on=['id'],how='left')
test_data = pd.merge(test_data,test_recieve_phone_count,on=['id'],how='left')
test_data = pd.merge(test_data,test_recieve_phone_nunique,on=['id'],how='left')

test_data = pd.merge(test_data,test_user_is_double_,on=['id'],how='left')
test_data = pd.merge(test_data,test_user_is_0_0_0,on=['id'],how='left')
test_data = pd.merge(test_data,test_user_is_1_1_1,on=['id'],how='left')
test_data = pd.merge(test_data,test_user_is_0000_00_00,on=['id'],how='left')
test_data = pd.merge(test_data,test_user_is_0001_1_1,on=['id'],how='left')
test_data = pd.merge(test_data,test_user_is_hou_in,on=['id'],how='left')

In [23]:
test_data['test_na_num'] = test_data.isnull().sum(axis=1)

# create "the_same_idcard"  

auth_idcard = list(test_data['id_card_x'])
user_idcard = list(test_data['id_card_y'])

idcard_result = []
for indexx, uu in enumerate(auth_idcard):
    
    if (str(auth_idcard[indexx])=='nan') and (str(user_idcard[indexx])=='nan'):
        idcard_result.append(0)
        
    elif (str(auth_idcard[indexx])!='nan') and (str(user_idcard[indexx])=='nan'):
        idcard_result.append(1)
        
    elif (str(auth_idcard[indexx])=='nan') and (str(user_idcard[indexx])!='nan'):
        idcard_result.append(2)
        
    else:
        ttt1 = str(auth_idcard[indexx])[0] + str(auth_idcard[indexx])[-1]
        ttt2 = str(user_idcard[indexx])[0] + str(user_idcard[indexx])[-1]
        
        if ttt1 == ttt2:
            idcard_result.append(3)
        if ttt1 != ttt2:
            idcard_result.append(4)
            
test_data['test_data_same_id_card'] = idcard_result


# 判断当前的手机号(test_auth)是否在手机号列表中，并剔除手机号列表
test_data['test_data_exist_phone'] = test_data.apply(lambda x: x['phone'] in x['test_bankcard_phone_list'], axis=1)
test_data['test_data_exist_phone'] = test_data['test_data_exist_phone']*1
test_data = test_data.drop(['test_bankcard_phone_list'], axis=1)

test_data['test_data_diff_day'] = test_data.apply(lambda x: (x['appl_sbm_tm'] - x['auth_time']).days, axis=1) # 贷款提交时间-认证时间
test_data['test_data_age'] = test_data.apply(lambda x: (x['appl_sbm_tm'] - x['birthday']).days / 365, axis=1) # 用户年龄

test_data['test_data_day_order_max'] = test_data.apply(lambda x: (x['appl_sbm_tm'] - x['test_order_time_max']).days, axis=1)
test_data = test_data.drop(['test_order_time_max'], axis=1)

test_data['test_data_day_order_min'] = test_data.apply(lambda row: (row['appl_sbm_tm'] - row['test_order_time_min']).days,axis=1)
test_data = test_data.drop(['test_order_time_min'],axis=1)

In [24]:
test_data = test_data.drop(['appl_sbm_tm','id','id_card_x','auth_time','phone','birthday','hobby','id_card_y'],axis=1)
test_data['target'] = -1 

In [25]:
test_data.to_csv('test_data_all_1117.csv', encoding='gbk', index=False)

In [26]:
train_test_data = pd.concat([train_data, test_data], axis=0, ignore_index = True)

In [27]:
train_test_data.to_csv('train_test_data_all_1117.csv', encoding='gbk', index=False)

In [29]:
pd.set_option('max_rows', 200)

In [30]:
train_test_data.dtypes

account_grade                        object
credit_score                        float64
degree                               object
hour                                  int64
income                               object
industry                             object
merriage                             object
month                                 int64
na                                    int64
overdraft                           float64
qq_bound                             object
quota                               float64
sex                                  object
target                                int64
test_auth_id_card_na                float64
test_auth_phone_na                  float64
test_bankcard_bank_name_count       float64
test_bankcard_bank_name_nunique     float64
test_bankcard_card_type_nunique     float64
test_bankcard_phone_nunique         float64
test_credit_can_use                 float64
test_credit_can_use_ratio           float64
test_credit_score_inverse       

In [31]:
train_test_data.fillna(0, inplace=True)

## 对下面几个变量进行独热编码
dummy_fea = ['sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound', 'account_grade', 'industry']
dummy_df = pd.get_dummies(train_test_data.loc[:, dummy_fea])

# 列合并独热编码的变量
train_test_data = pd.concat([train_test_data, dummy_df], axis=1)
train_test_data = train_test_data.drop(dummy_fea, axis=1)

## 分开训练数据和测试数据
train_data_new = train_test_data.iloc[:train_data.shape[0], :]
test_data_new = train_test_data.iloc[train_data.shape[0]:, :]

## 训练数据和测试数据都踢除target变量
train_data_new_x = train_data_new.drop(['target'], axis=1)
test_data_new_x = test_data_new.drop(['target'], axis=1)


In [32]:
## 
predict_result, modelee = xgb_feature(train_data_new_x, train_data_new['target'].values, test_data_new_x, None)

In [36]:
ans = pd.read_csv('AI_Risk_data_Btest_V2.0/test_list.csv', parse_dates = ['appl_sbm_tm'])

ans['PROB'] = predict_result
ans = ans.drop(['appl_sbm_tm'], axis=1)
minmin, maxmax = min(ans['PROB']), max(ans['PROB'])
ans['PROB'] = ans['PROB'].map(lambda x:(x-minmin)/(maxmax-minmin))
ans['PROB'] = ans['PROB'].map(lambda x:'%.4f' % x)

ans.to_csv('1117_result_test.csv',index=None)